# ChipArmour Bootloader Example Demo

## Unarmoured Bootloader

This notebook will explore how glitch attacks can be used to bypass a simplified secure bootloader implementing an asychronous crypto signing system for security. It will then demonstrate how ChipArmour can be integrated into the bootloader to protect against these attacks. 

The firmware is located in `examples/image_verification` and works as follows:

1. Check if firmware needs to be updated
2. Run a hash on the firmware
3. Check the hash using the public key to verify that it was correctly signed
4. If the signature is correct, load the new firmware and boot. Otherwise, boot the old firmware.

Looking at the `image` data, we can see that the image doesn't have the correct signature:

```C
image_t image = {
    "CA Demo Image", /* Name of image */
    {1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16}, /* Image data (fake) */
    256,   /* Length of image in bytes */
    //0x4C6509CC /* Correct Image signature */
    0xDEADBEEF   /* Incorrect image signature */
}```

Our goal will be to bypass the security functions of the bootloader and get this image running in spite of it not having the correct signature. If you've never done clock glitching before, you may want to run through ChipWhisperer's Fault_1 tutorial, which introduces clock glitching and uses it to skip instructions running on a target. An online mirror can be found here: https://chipwhisperer.readthedocs.io/en/latest/tutorials/fault_1-openadc-cwlitearm.html#tutorial-fault-1-openadc-cwlitearm

**This bootloader does not contain any cryptographic functions. It is not a cryptographically secure bootloader.**

This attack will use the ChipWhisperer firmware system and has been developed using the ChipWhisperer-Lite with Arm target platform. Firmware can be built as follows, with the `EXTRA_OPTS=IMAGE_UNARMOURED` option building the unarmoured firmware:

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'

In [28]:
%%bash -s "$PLATFORM"
make PLATFORM=$1 EXTRA_OPTS=IMAGE_UNARMOURED

rm -f -- image-demo-CWLITEARM.hex
rm -f -- image-demo-CWLITEARM.eep
rm -f -- image-demo-CWLITEARM.cof
rm -f -- image-demo-CWLITEARM.elf
rm -f -- image-demo-CWLITEARM.map
rm -f -- image-demo-CWLITEARM.sym
rm -f -- image-demo-CWLITEARM.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- image.s ../../../src/chiparmour.s stm32f3_hal.s stm32f3_hal_lowlevel.s stm32f3_sysmem.s
rm -f -- image.d ../../../src/chiparmour.d stm32f3_hal.d stm32f3_hal_lowlevel.d stm32f3_sysmem.d
rm -f -- image.i ../../../src/chiparmour.i stm32f3_hal.i stm32f3_hal_lowlevel.i stm32f3_sysmem.i
.
-------- begin --------
arm-none-eabi-gcc.exe (GNU Tools for Arm Embedded Processors 8-2019-q3-update) 8.3.1 20190703 (release) [gcc-8-branch revision 273027]
Copyright (C) 2018 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling C: image.c
arm-none-eabi-gcc -c -mcpu=cortex-m4 -I. -D

Next, we'll connect to the ChipWhisperer-Lite:

In [2]:
%run "Helper_Scripts/Setup_Generic.ipynb"

Serial baud rate = 38400


And program the newly built firmware onto the device:

In [29]:
cw.program_target(scope, prog, "image-demo-CWLITEARM.hex")
scope.clock.adc_src = "clkgen_x1"

Serial baud rate = 115200
Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 5351 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 5351 bytes
Serial baud rate = 38400


Next, let's take a look at the firmware, located in image.c:

```C
int main(void)
{
    //Check if fw update pending, apply if so
    platform_init();
    init_uart();
    trigger_setup();
    #ifdef IMAGE_UNARMOURED
        checkfwupdate_original();
    #elif defined(IMAGE_ARMOURED)
    //Check if fw update pending, apply if so
        checkfwupdate_armoured();
    #endif
    
    //No firmware update - start regular operations
    rtos_init();
    
    while(1){
        rtos_loop();
    }
}
```

And then into `checkfwupdate_original()`:

```C
int checkfwupdate_original(void)
{
    //Flag indicates new firmware file present
    if(bootloader_flag == FLAG_PENDING_UPDATE){
        
        //Check signature matches proposed hashes
        if (validate_sigature(some_hash_function(image.image_data, image.image_data_len),
                              image.signature, manf_public_key)) {
            
            trigger_low();
            boot_new_image(&image);
        } else {
            //signature failed
            trigger_low();
            bootloader_flag = 0;
        }
    }
    
    return 0;
}
```

```C
/**
 Original validation function which is vulnerable to FI.
 */
int validate_sigature(uint32_t short_hash, uint32_t signature, uint8_t * pub_key)
{
    trigger_high();
    if (some_crypto_function(short_hash, pub_key) == signature){
        return 1;
    } else {
        return 0;
    }
}
```

As a start, let's focus on validate signature, since that looks like a pretty vulnerable part of the code and reducing the range we need to check will save a lot of time on the attack. If we can get target to skip into `boot_new_image()` in `checkfwupdate_original()`, we'll have bypassed the signature check. We're using an IO pin here that we set to trigger off of, but it's important to understand that there are other ways to trigger that can be used in a real attack. For example, we could trigger off of a reset pin, a serial data line, or even use a pattern match on a recorded power trace.

We'll next need to find a way to determine whether our glitch was successful. We can use the fact that `boot_new_image()` prints the name of the image after the check has completed. Thus, if we see `"CA Demo Image"`in the response from the bootloader, we'll know we've broken the bootloader:

```C
image_t image = {
    "CA Demo Image", /* Name of image */
    {1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16}, /* Image data (fake) */
    256,   /* Length of image in bytes */
    //0x4C6509CC /* Correct Image signature */
    0xDEADBEEF   /* Incorrect image signature */
}```

```C
 void boot_new_image(image_t * image)
 {
     puts("Booting image ");
     puts(image->image_name);
     puts("\n");
     
     //Here would go code to actually do the real stuff
 }
```

We'll define a function to check if a given response corresponds to our firmware booting:

In [4]:
def glitch_success(resp):
    return "CA Demo Image" in resp

Next, we'll run the firmware without glitches to confirm that it won't boot the new firmware:

In [30]:
target.flush()
scope.io.hs2 = "clkgen"
scope.arm()
reset_target(scope)
ret = scope.capture()
if ret:
    print("Scope capture timed out")
response = target.read(timeout=10)
print(response)
print(scope.adc.trig_count)
print(glitch_success(response))

 RTOS Booted!


88
False


As expected, our image isn't booted and `glitch_success()` returns `False`. If you'd like, try replacing the signature for `image` and verify that `glitch_success()` will return `True`.

One additional piece of information we've gained from this is how long the image verification takes: 88 clock cycles. We can use this information to limit the area we try to glitch. If we haven't gotten a success after scanning all 88 clock cycles, we'll move on and try some additional settings.

Setting the ChipWhisperer up to glitch can be accomplished in a few lines of Python. We're using glitch settings that gave good results on other attack on this device:

In [31]:
#clock glitching
from collections import namedtuple
scope.io.glitch_hp=False
scope.glitch.clk_src = "clkgen"
scope.glitch.output = "clock_xor"
scope.glitch.trigger_src = "ext_single"
scope.io.hs2 = "glitch"

Range = namedtuple('Range', ['min', 'max', 'step'])
width_range = Range(-10.15, -7, 1)
scope.glitch.offset_fine = 0
#scope.glitch.offset = -39.84
offset_range = Range(-38, -37, 0.4)

print(scope.glitch)

clk_src     = clkgen
width       = -8.984375
width_fine  = 0
offset      = -37.890625
offset_fine = 0
trigger_src = ext_single
arm_timing  = after_scope
ext_offset  = 39
repeat      = 1
output      = clock_xor



Next, we have our glitch loop. As mentioned before, we'll scan the glitch from clock cycles 0 to 88. We'll also print when we get a successful glitch, as well as record settings used for successful glitches. In addition, we'll repeat each glitch `sample_size` times, since glitches often don't work 100% of the time, even with fine-tuned settings.

In [32]:
from collections import namedtuple
from tqdm import tnrange, trange

scope.glitch.width = width_range.min
scope.glitch.offset = offset_range.min
attack1_data = []
ext_offset_range = Range(0, 88, 1)
sample_size = 5
scope.glitch.repeat = 1

while scope.glitch.offset < offset_range.max:
    while scope.glitch.width < width_range.max:
        for j in trange(ext_offset_range.min, ext_offset_range.max, ext_offset_range.step, leave=False):
            successes = 0
            scope.glitch.ext_offset = j
            for i in range(sample_size):
                scope.arm()
                reset_target(scope)
                ret = scope.capture()
                if ret:
                    print('Timeout happened during acquisition')

                response = target.read(timeout = 10)

                # for table display purposes
                success = glitch_success(response)
                if success:
                    print("Success")
                    successes += 1
                if "RTOS" not in response:
                    print("Crashed")

            attack1_data.append([scope.glitch.width, scope.glitch.offset, successes/sample_size, scope.glitch.ext_offset, repr(response)])
            # run aux stuff that should happen after trace here
        scope.glitch.width += width_range.step
    scope.glitch.offset += offset_range.step
    scope.glitch.width = width_range.min
    print(scope.glitch)
print("Done glitching")

 31%|█████████████████████████▏                                                        | 27/88 [00:16<00:38,  1.59it/s]

Crashed


 41%|█████████████████████████████████▌                                                | 36/88 [00:22<00:32,  1.59it/s]

Crashed


 60%|█████████████████████████████████████████████████▍                                | 53/88 [00:33<00:22,  1.59it/s]

...

clk_src     = clkgen
width       = -10.15625
width_fine  = 0
offset      = -37.5
offset_fine = 0
trigger_src = ext_single
arm_timing  = after_scope
ext_offset  = 87
repeat      = 1
output      = clock_xor



clk_src     = clkgen
width       = -10.15625
width_fine  = 0
offset      = -37.109375
offset_fine = 0
trigger_src = ext_single
arm_timing  = after_scope
ext_offset  = 87
repeat      = 1
output      = clock_xor



 86%|██████████████████████████████████████████████████████████████████████▊           | 76/88 [00:47<00:07,  1.59it/s]

Crashed


clk_src     = clkgen
width       = -10.15625
width_fine  = 0
offset      = -36.71875
offset_fine = 0
trigger_src = ext_single
arm_timing  = after_scope
ext_offset  = 87
repeat      = 1
output      = clock_xor

Done glitching


Next, we'll print out the settings for any successful glitches we've had:

In [33]:
for row in attack1_data:
    if row[2] > 0:
        print(row)

[-10.15625, -37.890625, 0.2, 53, "'\\x00RTOS Booted!\\n\\n'"]
[-8.984375, -37.890625, 0.4, 53, "'\\x00Booting image \\nCA Demo Image\\n\\n\\nRTOS Booted!\\n\\n'"]
[-7.8125, -37.890625, 0.2, 53, "'\\x00RTOS Booted!\\n\\n'"]


You should find that the bootloader was quite susceptible to glitching in a single spot. There's not much assembly to look at here, so it's pretty easy to find a good candiate for the vulnerability:

```asm
 800028a:	f7ff ffe1 	bl	8000250 <validate_sigature>
 800028e:	4605      	mov	r5, r0
 8000290:	b138      	cbz	r0, 80002a2 <checkfwupdate_original+0x36>
                              image.signature, manf_public_key)) {
            
            trigger_low();
 8000292:	f000 f8c4 	bl	800041e <trigger_low>
            boot_new_image(&image);
 8000296:	f104 000c 	add.w	r0, r4, #12
 800029a:	f7ff ffc7 	bl	800022c <boot_new_image>
            bootloader_flag = 0;
```

As we can see, if the `cbz` (compare and branch on zero) instruction is skipped, or if the value in `r0` is corrupted, we'll bypass the security on the bootloader and be able to run our incorrectly signed firmware. This is just a small subset of the code as well: additional vulnerabilities may exist elsewhere.

Next, we'll take a look at the armoured version of the code.

## Armoured Bootloader

### Integrating ChipArmour

#### HAL Requirements

To implement all of ChipArmour's safety features, the following functions must be implemented:

```C
//Function called when a glitch attempt is detected.
//For example, you may want to jump to a fault handle function
void _ca_panic(void);

//Initialize MPU, RNG
// Only required if using MPU and jitter features
void ca_init(void);
```

#### Replacing If Statements

Before we try our hand at glitching, let's first look at how to incorporate ChipArmour into the bootloader. Overall, the goal is to move parts of the code vulnerable to glitching into the ChipArmour library so that glitch mitigation can be handled in a central location. Our first goal will be to get rid of any `if` statements, since they're often fairly trivial to glitch past. ChipArmour provides `ca_compare_u32_eq` to accomplish this. As we can see from the prototype and its documentation, it functions like a wrapper around an `if` statement:

```C
/**
    Compare two uint32_t numbers, and call a function if they are the same or
    another function if they are different.
    
    op1: Operand 1
    op2: Operand 2
    equal_function: Function of type 'void func(void *)' that will be called
                    with argument equal_func_param if op1 == op2. Null if you
                    don't need a function called on match.
    unequal_function: Function of type 'void func(void *)' that will be called
                    with argument unequal_func_param if op1 != op2. Null if you
                    don't need a function called on differ.
*/

static inline ca_return_t ca_compare_u32_eq( uint32_t op1, 
                               uint32_t op2,
                               ca_fptr_voidptr_t equal_function,
                               void * equal_func_param,
                               ca_fptr_voidptr_t unequal_function,
                               void * unequal_func_param)
```

So instead of:

```C
if (bootloader_flag == FLAG_PENDING_UPDATE) {
    // update_firmware(update_firmware_param);
} else {
    // nothing(nothing_param);
}
```

into something along the lines of:

```C
ca_compare_u32_eq(bootloader_flag, FLAG_PENDING_UPDATE, //if (bootloader_flag == FLAG_PENDING_UPDATE) {
                  update_firmware, update_firmware_param, //update_firmware(update_firmware_param); }
                  nothing, nothing_param); // else { nothing(nothing_param);}
```

Looking at `checkfwupdate_armoured()`, that's exactly what we see:

```C
    ca_compare_u32_eq(bootloader_flag,
                      FLAG_PENDING_UPDATE,
                      fw_update_stage1,
                      (void *)&image,
                      fw_update_stage1_failed,
                      (void *)&image);
```

Looking at `fw_update_stage1()` and `fw_update_stage1_failed()`, we can see that replace what was previously in each unarmoured `if` branch:

```C
void fw_update_stage1(void * image)
{
    uint32_t hash = some_hash_function(((image_t *)image)->image_data, ((image_t *)image)->image_data_len);
    
    //Possible new image - first we calculate hash of image data, then check signature
    uint32_t crypto_ret = 0;
    ca_compare_func_eq(wrapper_some_crypto_function,
                       (void *)&hash,
                       (uint8_t *) (&crypto_ret),
                       (uint8_t *) (&((image_t *)image)->signature),
                       4,
                       (ca_fptr_voidptr_t)boot_new_image_armoured,
                       (void *)image,
                       fw_update_stage2_failed,
                       (void *)image);
    
}

void fw_update_stage1_failed(void * image)
{    
    //Flag not set - boot as normal
    bootloader_flag = 0;
}
```

It's important to what safety is being gained by calling this ChipArmour function. We're not ensuring that either branch is called here, just that neither branch is **incorrectly** called. There's additional features in ChipArmour that can help protect against out of order execution. This will be discussed later.

There's nothing too different happending if stage 1 fails, so let's continue on with `fw_update_stage1()`. From the original code, we know that we need to:

1. Calculate the hash of the image
1. Calculate the signature of the image
1. Boot the old firmware if the signature is wrong and boot the new firmware if the signature is correct

Again, we won't focus on ensuring code is called yet, so `some_hash_function()` will be left alone for now. The signature calculation and signature check are much bigger problems. We'll need to replace it in a similar way to the first `if`, this time using `ca_compare_func_eq()`. It's a little more complicated than the first `ca_compare` function, but can still be integrated with some simple wrappers and an extra variable.

First, looking at the prototype for `ca_compare_func_eq()`:

```C
/**************************************************************************
 Signature verification functions / macros
 **************************************************************************/

/**
    Signature verification: compares the result of a function call with some
    magic value, and calls one of two functions in response.

    get_value_func: function of the type 'void func(void *, uint8_t *array)
                    that will be called and compared to expected_value_array.
                    The result from func() that will be compared is array.
    get_value_func_param: void * passed to get_value_func
    get_value_func_return: Memory from get_value_func
    expected_value_array: Value to be compared with the return of get_value_func()
    expected_value_len: Length of expected_value_array
    equal_function: Called if the result of get_value_func() and expected_value_array are the same.
                    Of the form 'void func(void *)
    equal_func_param: void * passed to equal_function()
    unequal_function: Called if the result of get_value_func() and expected_value_array are unequal.
                    Of the form 'void func(void *)
    unequal_func_param: void * passed to unequal_function()
*/
ca_return_t ca_compare_func_eq( ca_fptr_voidptr_array_t    get_value_func,
                             void *                     get_value_func_param,
                             uint8_t *                  get_value_func_return,
                             uint8_t *                  expected_value_array,
                             uint32_t                   expected_value_len,
                             ca_fptr_voidptr_t           equal_function,
                             void *                     equal_func_param,
                             ca_fptr_voidptr_t          unequal_function,
                             void *                     unequal_func_param)
```

We can see that it's similar to `ca_compare_u32_eq()`, but with a few key differences:

* `op1` is now a function of the form 'void func(void *, uint8_t *array)'
    * `some_crypto_function()` doesn't follow this form, so we'll need a wrapper function
* We'll need to pass in some memory to hold the result of `some_crypto_function()`
* `op2` is now an array (`expected_value_array`). As such, we also need to pass in a length (`expected_value_len`)

The crypto wrapper is fairly simple. We just need to call `some_crypto_function()`, then copy its return into `output`:

```C
void wrapper_some_crypto_function(void * input, uint8_t * output)
{
    uint32_t expected;
    expected = some_crypto_function(*((uint32_t *)input), manf_public_key);
    
    *((uint32_t *)output) = expected;
}
```

`expected_value_array` is 4 bytes long, so we can get the memory from a `uint32_t`:

```C
uint32_t crypto_ret = 0;
```

and we're ready to call `ca_compare_func_eq()`.

Once everything is put into place, `fw_update_stage1()` looks as follows:

```C
/**
 Step 1: Magic flag set, check signature ok by public key.
 */
void fw_update_stage1(void * image)
{
    uint32_t hash = some_hash_function(((image_t *)image)->image_data, ((image_t *)image)->image_data_len);
    
    //Possible new image - first we calculate hash of image data, then check signature
    uint32_t crypto_ret = 0;
    ca_compare_func_eq(wrapper_some_crypto_function,
                       (void *)&hash,
                       (uint8_t *) (&crypto_ret),
                       (uint8_t *) (&((image_t *)image)->signature),
                       4,
                       (ca_fptr_voidptr_t)boot_new_image_armoured,
                       (void *)image,
                       fw_update_stage2_failed,
                       (void *)image);
    
}
```

With that done, we're ready to move onto other ChipArmour features.

#### CA_State_Machine

Another important feature of ChipArmour is its state machine. It works as follows: the user first initializes the state machine. Next, the user places calls to `ca_state_machine(N)` in their code. Each time `ca_state_machine()` is called, N must be incremented by 1, beginning with 1 (after initialization), otherwise the function will panic. This allows a user to ensure code is executed in a set order. To make things more clear in the context of glitching, let's take a look at an example and consider the effect that glitching has on it:

```C
#include "chiparmour.h"

void call_thrid(void)
{
    ca_state_machine(3);
    //important code...
}

void call_second(void)
{
    ca_state_machine(2);
    //important code...
}

void call_first(void)
{
    ca_state_machine(1);
    //important code...
}

int main(void)
{
    ca_state_machine(CA_INIT);
    
    call_first();
    call_second();
    call_third();
    ca_state_machine(4);
    return 0;
}
```

During normal execution, we can see that everything proceeds fine: the state machine is initialized and each call to `ca_state_machine()` passes a number one higher than the last. Next, let's take a look at a common glitch scenario: an attacker wants to skip one of these function calls. Let's say `call_second()` is skipped. This means `ca_state_machine(2)` is also skipped. Since the sequence of 1, 2, 3... is broken, the call to `ca_state_machine(3)` will cause the program to panic. In the context of a more complicated program, this can also protect against attackers directly calling functions. We can see this function in use throughout the armoured portion of `image.c`:

```C
int checkfwupdate_armoured(void)
{
    ca_state_machine(CA_STATE_INIT); //<----
    trigger_high();
    //Flag indicates new firmware file present
    ca_compare_u32_eq(bootloader_flag,
                      FLAG_PENDING_UPDATE,
                      fw_update_stage1,
                      (void *)&image,
                      fw_update_stage1_failed,
                      (void *)&image);
    
    trigger_low();
    ca_state_machine(3); //<----
    
    return 0;
}

void fw_update_stage1(void * image)
{    
    ca_state_machine(1); //<----
    uint32_t hash = some_hash_function(((image_t *)image)->image_data, ((image_t *)image)->image_data_len);
    
    //Possible new image - first we calculate hash of image data, then check signature
    uint32_t crypto_ret = 0;
    ca_compare_func_eq(wrapper_some_crypto_function,
                       (void *)&hash,
                       (uint8_t *) (&crypto_ret),
                       (uint8_t *) (&((image_t *)image)->signature),
                       4,
                       (ca_fptr_voidptr_t)boot_new_image_armoured,
                       (void *)image,
                       fw_update_stage2_failed,
                       (void *)image);
    
}

/**
 Firmware update function, called once we know all images are OK.
 */
 void boot_new_image_armoured(image_t * image)
 {
     //CA_ROP_CHECK_VALID_RETURN(boot_new_image);
     
     ca_state_machine(2); //<---
     
     //ca_unmangle_var(image, 1);
     
     puts("Booting image ");
     puts(image->image_name);
     puts("\n");
     
     //Here would go code to actually do the real stuff
 }


```

We can see from examining the other parts of branches that we need to make sure the state machine of other branches lines up as well. As such, there's a tradeoff here: having more calls to the state machine will mean more protection against out of order execution at the cost of greater complexity in the state machine. For example, from above we can see that `fw_update_stage1()` has two `ca_state_machine()` calls. This means that we'll need to replicate that in the `fw_update_stage1_failed()` branch as well to prevent the stage machine call afterwords from panicking:

```C
void fw_update_stage1_failed(void * image)
{
    //Prevent out of order function calls
    //Needs to be done twice to match the other branch
    ca_state_machine(1);
    ca_state_machine(2);

    //Flag not set - boot as normal
    bootloader_flag = 0;
}
```

#### Additional Features

Some additional glitch mitigation features of ChipArmour include armoured limit functions:

```C
/**
    Take an input value and ensure it falls within the given limits, by 
    returning the limited value.
*/
static inline uint32_t ca_limit_u32(uint32_t input, uint32_t min, uint32_t max)
```

Armoured integer returns:

```C
/**
    Returns a 32-bit unsigned int, but after a random delay to assist with 
    FI armouring.
*/
ca_uint32_t ca_ret_u32(uint32_t value);
```

Landmines (perform comparisons that should always succeed, panic if they fail):

```C
/** 
  Jumps to the panic function if one of two comparisons fail.
  */

#define ca_landmine() { if(_ca_sram_FEED7431 != 0xFEED7431){ca_panic();} \
                        if(_ca_flash_55A88519 != 0x55A88519){ca_panic();} \
                        if(_ca_sram_FEED7431 == _ca_flash_55A88519){ca_panic();} }
```

As well as various test functions for ensuring ChipArmour is properly setup.

#### Upcoming Features

As of March 2020, ChipArmour is still in an early Alpha state, with many features and improvements planned in the future. These include:

* Validating return locations to prevent functions from being called from unexpected other functions
* MPU support
    * Locking and unlocking the MPU
    * Constructs to allow variables to be easily placed in MPU protected memory
* Additional type support
* Random jitter to throw off glitch timing

### Glitching Armoured Bootloader

Now that we know how ChipArmour is integrated into the bootloader, let's try some glitches and see how it holds up. First, we'll rebuild the firmware, this time using armoured code:

In [16]:
%%bash -s "$PLATFORM"
make PLATFORM=$1 EXTRA_OPTS=IMAGE_ARMOURED

rm -f -- image-demo-CWLITEARM.hex
rm -f -- image-demo-CWLITEARM.eep
rm -f -- image-demo-CWLITEARM.cof
rm -f -- image-demo-CWLITEARM.elf
rm -f -- image-demo-CWLITEARM.map
rm -f -- image-demo-CWLITEARM.sym
rm -f -- image-demo-CWLITEARM.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- image.s ../../../src/chiparmour.s stm32f3_hal.s stm32f3_hal_lowlevel.s stm32f3_sysmem.s
rm -f -- image.d ../../../src/chiparmour.d stm32f3_hal.d stm32f3_hal_lowlevel.d stm32f3_sysmem.d
rm -f -- image.i ../../../src/chiparmour.i stm32f3_hal.i stm32f3_hal_lowlevel.i stm32f3_sysmem.i
.
-------- begin --------
arm-none-eabi-gcc.exe (GNU Tools for Arm Embedded Processors 8-2019-q3-update) 8.3.1 20190703 (release) [gcc-8-branch revision 273027]
Copyright (C) 2018 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling C: image.c
arm-none-eabi-gcc -c -mcpu=cortex-m4 -I. -D

And then upload this onto the target:

In [17]:
cw.program_target(scope, prog, "image-demo-CWLITEARM.hex")
scope.clock.adc_src = "clkgen_x1"

Serial baud rate = 115200
Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 7523 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 7523 bytes
Serial baud rate = 38400


And again we'll run a test without glitching:

In [19]:
scope.io.hs2 = "clkgen"

target.flush()
scope.arm()
reset_target(scope)


ret = scope.capture()
if ret:
    print("Scope capture timed out")
response = target.read(timeout=10)
print(response)
print(scope.adc.trig_count)
print(glitch_success(response))

 RTOS Booted!


3165
False


And like before, we'll try some glitching:

In [21]:
#clock glitching
from collections import namedtuple
scope.io.glitch_hp=False
scope.glitch.clk_src = "clkgen"
scope.glitch.output = "clock_xor"
scope.glitch.trigger_src = "ext_single"
scope.io.hs2 = "glitch"

Range = namedtuple('Range', ['min', 'max', 'step'])
width_range = Range(-10.15, -7, 1)
scope.glitch.offset_fine = 0
#scope.glitch.offset = -39.84
offset_range = Range(-38, -37, 0.4)


print(scope.glitch)

clk_src     = clkgen
width       = -10.15625
width_fine  = 0
offset      = -37.890625
offset_fine = 0
trigger_src = ext_single
arm_timing  = after_scope
ext_offset  = 1268
repeat      = 1
output      = clock_xor



In [22]:
from collections import namedtuple
from tqdm import tnrange
f = open("log.txt", "w")

scope.glitch.width = width_range.min
scope.glitch.offset = offset_range.min
attack1_data = []
ext_offset_range = Range(0, 3165, 1)
sample_size = 3
scope.glitch.repeat = 1

while scope.glitch.offset < offset_range.max:
    while scope.glitch.width < width_range.max:
        for j in tnrange(ext_offset_range.min, ext_offset_range.max, ext_offset_range.step, leave=False):
            successes = 0
            scope.glitch.ext_offset = j
            for i in range(sample_size):
                scope.arm()
                reset_target(scope)
                ret = scope.capture()
                if ret:
                    print('Timeout happened during acquisition')

                response = target.read(timeout = 10)

                # for table display purposes
                success = glitch_success(response)
                if success:
                    print("Success")
                    print(response)
                    successes += 1
                if "RTOS" not in response:
                    print("Crashed")
                #f.write("___________________")
                #f.write(response)
                #f.write("-------------------")

            attack1_data.append([scope.glitch.width, scope.glitch.offset, successes/sample_size, scope.glitch.ext_offset, repr(response)])
            # run aux stuff that should happen after trace here
        scope.glitch.width += width_range.step
    scope.glitch.offset += offset_range.step
    scope.glitch.width = width_range.min
    print(scope.glitch)
print("Done glitching")

Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed


Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed


Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed


Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed


Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed


Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
clk_src     = clkgen
width       = -10.15625
width_fine  = 0
offset      = -37.109375
offset_fine = 0
trigger_src = ext_single
arm_timing  = after_scope
ext_offset  = 3164
repeat      = 1
outp

Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed


Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed


Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
clk_src     = clkgen
width       = -10.15625
width_fine  = 0
offset      = -36.71875
offset_fine = 0
trigger_src = ext_single
arm_timing  = after_scope
ext_offset  = 3164
repeat      = 1
output      = clock_xor

Done glitching


In [23]:
for row in attack1_data:
    if row[2] > 0:
        print(row)

You should find that the new armoured firmware stood up much better to a simple glitch attack like this.

In [9]:
f.close()

In [15]:
scope.dis()
target.dis()